# 이미지 분석

In [1]:
import easyocr
import numpy as np

### OCR 클래스

In [2]:
import easyocr
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from PIL import ImageFont, ImageDraw, Image

from konlpy.tag import Kkma

class Ocr:
    def __init__(self, img):
        self.img = img

    def text_detect(self):
        # img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        
        reader = easyocr.Reader(['ko', 'en'], gpu = True)
        result = reader.readtext(self.img)

        # img = cv2.resize(img_cv2, (320, 448))
        image = Image.fromarray(self.img)
        font = ImageFont.truetype('C:/Users/tjoeun/AppData/Local/Microsoft/Windows/Fonts/NanumGothicBold.ttf', 10)
        draw = ImageDraw.Draw(image)
        np.random.seed(42)
        COLORS = np.random.randint(0, 255, size=(255, 3),dtype="uint8")
        for i in result :
            x = i[0][0][0] 
            y = i[0][0][1] 
            w = i[0][1][0] - i[0][0][0] 
            h = i[0][2][1] - i[0][1][1]

            color_idx = random.randint(0,255) 
            color = [int(c) for c in COLORS[color_idx]]

            draw.rectangle(((x, y), (x+w, y+h)), outline=tuple(color), width=2)
            draw.text((int((x + x + w) / 2) , y-2),str(i[1]), font=font, fill=tuple(color),)

        # plt.figure(figsize=(10,20))
        plt.imshow(image)
        plt.axis("off")
        plt.show()
        # print(result)

    def image_to_text(self):
        # 추출된 텍스트 영역을 이용하여 각각의 텍스트를 잘라내고 EasyOCR을 다시 적용
        reader = easyocr.Reader(['ko', 'en'], gpu=True)
        result = reader.readtext(self.img)
        
        origin_results = []
        roi_results = []
        roi_gray_results =[]
        roi_2_results =[]
        roi_equal_results =[]
        roi_dil_results =[]
        roi_ero_results =[]
        roi_canny_results =[]

        for (box, text, confidence) in result:
            try:
                origin_results.append(text)
            except:
                pass

            # 각 텍스트 박스의 좌표 추출
            (startX, startY) = box[0]
            (endX, endY) = box[2]
            
            # 좌표를 정수로 변환
            startX, startY, endX, endY = int(startX), int(startY), int(endX), int(endY)

            # print(startX, startY, endX, endY)
            # 좌표가 음수이거나 이미지 범위를 벗어나는 경우 무시
            if startX < 0 or startY < 0 or endX >= self.img.shape[1] or endY >= self.img.shape[0]:
                continue

            # 텍스트 영역 추출
            roi = self.img[startY:endY, startX:endX]

            ############################################# 이미지 전처리 ###################################################

            # 이미지를 그레이스케일로 변환
            roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

            # 적응형 이진화 수행
            roi_adaptive_thresh = cv2.adaptiveThreshold(roi_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 2)

            # 히스토그램 평탄화 수행
            roi_equalized = cv2.equalizeHist(roi_gray)

            # 팽창
            kernel = np.ones((3, 3), np.uint8)
            roi_dilation = cv2.dilate(roi, kernel, iterations=1)

            # 침식
            kernel = np.ones((3, 3), np.uint8)
            roi_erosion = cv2.erode(roi, kernel, iterations=1)

            # 캐니
            roi_canny = cv2.Canny(roi, 475, 500, apertureSize=3, L2gradient=True)

            ############################################# 텍스트 인식 ###################################################

            # 잘라낸 텍스트 영역에 EasyOCR을 적용
            roi_result = reader.readtext(roi)
            roi_gray_result = reader.readtext(roi_gray)
            roi_2_result = reader.readtext(roi_adaptive_thresh)
            roi_equal_result = reader.readtext(roi_equalized)
            roi_dil_result = reader.readtext(roi_dilation)
            roi_ero_result = reader.readtext(roi_erosion)
            roi_canny_result = reader.readtext(roi_canny)

            ############################################# 결과 출력 ###################################################

            # print("부분")
            # plt.imshow(roi)
            try:
                # print(f'Text: {roi_result[0][1]}, Confidence: {roi_result[0][2]}')
                roi_results.append(roi_result[0][1])
            except:
                # print("Not Detect")
                roi_results.append('None')
            # plt.axis('off')
            # plt.show()

            # print("그레이 스케일")
            # plt.imshow(roi_gray, cmap="gray")
            try:
                # print(f'Text: {roi_gray_result[0][1]}, Confidence: {roi_gray_result[0][2]}')
                roi_gray_results.append(roi_gray_result[0][1])
            except:
                # print("Not Detect")
                roi_gray_results.append('None')
            # plt.axis('off')
            # plt.show()

            # print("적응형 이진화")
            # plt.imshow(roi_adaptive_thresh, cmap="gray")
            try:
                # print(f'Text: {roi_2_result[0][1]}, Confidence: {roi_2_result[0][2]}')
                roi_2_results.append(roi_2_result[0][1])
            except:
                # print("Not Detect")
                roi_2_results.append('None')
            # plt.axis('off')
            # plt.show()

            # print("히스토그램 평탄화")
            # plt.imshow(roi_equalized)
            try:
                # print(f'Text: {roi_equal_result[0][1]}, Confidence: {roi_equal_result[0][2]}')
                roi_equal_results.append(roi_equal_result[0][1])
            except:
                # print("Not Detect")
                roi_equal_results.append('None')
            # plt.axis('off')
            # plt.show()

            # print("팽창")
            # plt.imshow(roi_dilation)
            try:
                # print(f'Text: {roi_dil_result[0][1]}, Confidence: {roi_dil_result[0][2]}')
                roi_dil_results.append(roi_dil_result[0][1])
            except:
                # print("Not Detect")
                roi_dil_results.append('None')
            # plt.axis('off')
            # plt.show()

            # print("침식")
            # plt.imshow(roi_erosion)
            try:
                # print(f'Text: {roi_ero_result[0][1]}, Confidence: {roi_ero_result[0][2]}')
                roi_ero_results.append(roi_ero_result[0][1])
            except:
                # print("Not Detect")
                roi_ero_results.append('None')
            # plt.axis('off')
            # plt.show()

            # print("캐니")
            # plt.imshow(roi_canny)
            try:
                # print(f'Text: {roi_canny_result[0][1]}, Confidence: {roi_canny_result[0][2]}')
                roi_canny_results.append(roi_canny_result[0][1])
            except:
                # print("Not Detect")
                roi_canny_results.append('None')
            # plt.axis('off')
            # plt.show()
        return origin_results, roi_results, roi_gray_results, roi_2_results, roi_equal_results, roi_dil_results, roi_ero_results, roi_canny_results


class Get_Nouns:
    def __init__(self, i2t_text):
        self.text = i2t_text

    def get_nouns(self):
        kkma = Kkma()
        nouns = kkma.nouns(str(self.text))
        return nouns

    def get_nouns_list(self):
        i2t_nouns = self.get_nouns()

        noun_list = []

        # 추출된 명사 출력
        for i2t_noun in i2t_nouns:
            if len(i2t_noun) > 1:
                noun_list.append(i2t_noun)

        # noun_list = list(set(noun_list))
        return noun_list
    
    def get_nouns_freq(self, act_nouns, ani_nouns, com_nouns, dra_nouns, hor_nouns, etc_nouns):
        self.act_nouns = act_nouns
        self.ani_nouns = ani_nouns
        self.com_nouns = com_nouns
        self.dra_nouns = dra_nouns
        self.hor_nouns = hor_nouns
        self.etc_nouns = etc_nouns

        nouns = self.get_nouns_list()
        
        act_freq = 0
        ani_freq = 0
        com_freq = 0
        dra_freq = 0
        hor_freq = 0
        etc_freq = 0

        # 입력한 단어들의 빈도수 확인
        try:
            for word in nouns:
                if word in self.act_nouns:
                    act_freq += 1
                elif word in self.ani_nouns:
                    ani_freq += 1
                elif word in self.com_nouns:
                    com_freq += 1
                elif word in self.dra_nouns:
                    dra_freq += 1
                elif word in self.hor_nouns:
                    hor_freq += 1
                else:
                    etc_freq += 1
        except:
            pass
        
        return act_freq, ani_freq, com_freq, dra_freq, hor_freq, etc_freq

### 장르별 단어 txt파일 불러오기

In [3]:
def read_words_from_file(file_path):
    words = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 줄바꿈 문자를 제거하고 단어를 리스트에 추가
            word = line.strip()
            words.append(word)
    return words

# 파일로부터 단어 리스트 불러오기
act_nouns = read_words_from_file('c:/3rd_project/data/act_nouns.txt')
ani_nouns = read_words_from_file('c:/3rd_project/data/ani_nouns.txt')
com_nouns = read_words_from_file('c:/3rd_project/data/com_nouns.txt')
dra_nouns = read_words_from_file('c:/3rd_project/data/dra_nouns.txt')
hor_nouns = read_words_from_file('c:/3rd_project/data/hor_nouns.txt')
etc_nouns = read_words_from_file('c:/3rd_project/data/etc_nouns.txt')

# Main

In [5]:
X = np.load('c:/3rd_project/data/images/movie/X_total_color.npy')
y = np.load('c:/3rd_project/data/images/movie/y_total_color.npy')

sample_img = X[7]
sample_gen = y[0]

img = sample_img
genre = sample_gen


## 이미지 텍스트 추출
ocr = Ocr(img)
# ocr.text_detect()
i2t_text = ocr.image_to_text()
# print(y[num])
gn = Get_Nouns(i2t_text)
nouns = gn.get_nouns_list()
print(gn.get_nouns_freq(act_nouns, ani_nouns, com_nouns, dra_nouns, hor_nouns, etc_nouns))

## 이미지 특징 추출


(13, 3, 2, 26, 0, 0)
